In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import timedelta
import matplotlib.pyplot as plt
from pandarallel import pandarallel
import time

In [2]:
pandarallel.initialize(shm_size_mb = 1000)

New pandarallel memory created - Size: 1000 MB
Pandarallel will run on 24 workers


In [3]:
clean_data_path = "wikiaves_registries.feather"

In [4]:
paths = ["wikiaves_authors_1-10000.pkl",
         "wikiaves_authors_10000-20000.pkl",
         "wikiaves_authors_20000-30000.pkl",
         "wikiaves_authors_30001-36809.pkl"
        ]

raw_data_list = []
for path in paths:
  raw_data_list.append(pd.read_pickle(path))
raw_data = pd.concat(raw_data_list).reset_index()#.drop_duplicates(subset=['id', 'user_id', 'data'])

In [5]:
time1 = time.time()
drop_cols = ['enviado', 'id_usuario', 'por', 'tipo']

rename_map = {"user_id": "author_id",
              "autor": "author_name",
              "coms": "comment_count",
              "data": "registry_date",
              "grande": "is_large",
              "id": "registry_id",
              "idMunicipio": "location_id",
              "is_questionada": "is_flagged",
              "likes": "like_count",
              "link": "registry_link",
              "local": "location_name",
              "perfil": "profile_name",
              "vis": "views_count",
              "hometown_ids": "home_location_id",
             }

clean_data = (raw_data#.sample(10000)
                  .assign(autor=lambda df: pd.Categorical(df['autor']))
                  .assign(coms=lambda df: pd.to_numeric(df['coms']))
                  .assign(data=lambda df: pd.to_datetime(df['data'], format='%d/%m/%Y', errors='coerce'))
                  .assign(grande=lambda df: df['grande'].astype(str) == "T")
                  .assign(id=lambda df: pd.to_numeric(df['id']))
                  .assign(idMunicipio=lambda df: pd.to_numeric(df['idMunicipio']))
                  .assign(is_questionada=lambda df: df['is_questionada'].astype(bool))
                  .assign(likes=lambda df: pd.to_numeric(df['likes']))
                  .assign(link=lambda df: df['link'].astype(str))
                  .assign(local=lambda df: pd.Categorical(df['local']))
                  .assign(perfil=lambda df: pd.Categorical(df['perfil']))
                  .assign(vis=lambda df: pd.to_numeric(df['vis']))
                  .assign(hometown_ids=lambda df: pd.to_numeric(df['hometown_ids']))
                  .rename(columns=rename_map)
                  .drop(columns=drop_cols)
             )
print(time.time() - time1)

28.0275399684906


In [ ]:
len(sp_data)

In [ ]:
sp_data = clean_data['sp'].parallel_apply(pd.Series)

In [ ]:
#time1 = time.time()
#raw_data = (raw_data.pipe(lambda df: df.join(, rsuffix='_sp'))
#                .drop(columns=['sp']))
#print(time.time() - tim

In [ ]:
#              "id_sp": "species_id",
#              "nome": "scientific_species_name",
#              "nvt": "popular_species_name",
#              "idwiki": "species_wiki_slug"

#                  .assign(id_sp=lambda df: pd.to_numeric(df['id_sp']))
#                  .assign(nome=lambda df: pd.Categorical(df['nome']))
#                  .assign(nvt=lambda df: pd.Categorical(df['nvt']))
#                  .assign(idwiki=lambda df: pd.Categorical(df['idwiki']))

In [6]:
clean_data.dtypes

author_id                    int64
author_name               category
comment_count                int64
registry_date       datetime64[ns]
is_large                      bool
registry_id                  int64
location_id                float64
is_flagged                    bool
like_count                   int64
registry_link               object
location_name             category
profile_name              category
sp                          object
views_count                  int64
home_location_id           float64
dtype: object

In [ ]:
clean_data.to_feather(clean_data_path)

In [ ]:
clean_data = pd.read_feather(clean_data_path)

In [ ]:
clean_data.dtypes